# EfficientNet
이 문서는 여러 글을 참고하여 만든 EfficientNet입니다.

## 참고
Jie Hu, Li Shen, Samuel Albanie, Gang Sun and, Enhua Wu, "Squeeze-and-Excitaion Networks",arxiv:1709.01507[https://arxiv.org/abs/1709.01507]
towardDataSience :Squeeze-and-Excitation Networks[https://towardsdatascience.com/squeeze-and-excitation-networks-9ef5e71eacd7]
Image Classification with EfficientNet: Better performance with computational efficiency[https://medium.com/analytics-vidhya/image-classification-with-efficientnet-better-performance-with-computational-efficiency-f480fdb00ac6]

In [ ]:
import collections
import math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras

In [ ]:
train = pd.read_csv('/path/to/train.csv')
test = pd.read_csv('/path/to/test.csv')

In [ ]:
# Data Preprocessing

train_img = train.iloc[:,3:].to_numpy().reshape(-1,28,28,1)
train_digit = train['digit']
train_letter = train['letter']
test_img = test.iloc[:,2:].to_numpy().reshape(-1,28,28,1)
test_letter = test['letter']

In [ ]:
train_img_norm = train_img / 255.0
test_img_norm = test_img / 255.0

In [ ]:
X_train = train_img_norm
y_train = train_digit.to_numpy()

X_test = test_img_norm

In [ ]:
from sklearn.model_selection import train_test_split

X_train_data, X_val_data, y_train_data, y_val_data = train_test_split(X_train, y_train, test_size = 0.2, random_state = 1004)

In [ ]:
def loss_plot(epochs, hist, list_of_metrics):
    
    fig, ax = plt.subplots(1,2,figsize = (12, 8))
    
    for i in range(len(ax)):
        ax[i].set_xlabel('Epochs')
        ax[i].set_ylabel('Value')
        
        for n in range(len(list_of_metrics)):
            if i == 0:
                y = hist[list_of_metrics[n]]
                ax[i].plot(epochs, y)
                ax[i].set_title('Loss')
                if n == 1:
                    break
            else:
                if n >= 2:
                    y = hist[list_of_metrics[n]]
                    ax[i].plot(epochs, y)
                    ax[i].set_title('Accuracy')
                    
    plt.show()

In [ ]:
from nn_block.EfficientNet.EfficientNet_parameters import Global_params, Block_params, CONV_KERNEL_INITIALIZER, DENSE_KERNEL_INITIALIZER

global_param = Global_params(dropout_rate=0.2, batch_norm=False, batch_norm_momentum=0.99,
                             batch_norm_epsilon=0.001, activation = 'relu')

Block_params_list = [
    Block_params(channels=128, output_filters=256, kernel_size = 3, strides=1, padding='same',
                 expand_ratio=3, se_ratio=0.25,num_repeat=9, block_number=2),
    Block_params(channels=256, output_filters=256, kernel_size = 3, strides=1, padding='same',
                expand_ratio=3, se_ratio=0.25,num_repeat=9, block_number=3),
    Block_params(channels=256, output_filters=256, kernel_size = 3, strides=1, padding='same',
                expand_ratio=3, se_ratio=0.25, num_repeat=1, block_number=4)
]

# MBConvBlock
- out : return the MBblock with seblocks.

# EfficientNet 

    
# **params**
- width_coefficient
- depth_coefficient
- default_resolution
- dropout_rate
- global_params
- block_params_list
- depth_divisor
- classes

# Function
- round_filters
- round_repeats

In [ ]:
from nn_block.EfficientNet.EfficientNet_utils import round_filters, round_repeats
from nn_block.EfficientNet.EfficientNet_Block import MBConvBlock

In [ ]:
def EfficientNet(width_coefficient,
                 depth_coefficient,
                 default_resolution,
                 dropout_rate,
                 global_params,
                 block_params_list,
                 depth_divisor=8,
                 classes=10,
                 ):

    input = layers.Input(shape=(default_resolution, default_resolution, 1))
    x = layers.Conv2D(128,
                      kernel_size=(3, 3),
                      strides=1,
                      padding='same',
                      kernel_initializer=CONV_KERNEL_INITIALIZER,
                      name = 'conv1')(input)
    if global_params.batch_norm:
        x = layers.BatchNormalization(momentum=global_params.batch_norm_momentum,
                                      epsilon=global_params.batch_norm_epsilon,
                                      name = 'batch_normalization1'
                                      )(x)
    x = layers.Activation(global_params.activation, name = 'activation1')(x)

    for i in range(len(block_params_list)):
        args = block_params_list[i]
        args = args._replace(channels=round_filters(args.channels, width_coefficient, depth_divisor),
                             output_filters=round_filters(args.output_filters, width_coefficient, depth_divisor),
                             num_repeat=round_repeats(args.num_repeat, depth_coefficient))

        x = MBConvBlock(global_params, args).out(x)
        x = layers.MaxPool2D((2, 2), 2, name = 'pool{}'.format(i))(x)

    # FC layers
    img_features = layers.Flatten()(x)
    img_features = layers.Dense(512, kernel_initializer=DENSE_KERNEL_INITIALIZER, name='fc_1')(img_features)
    img_features = layers.Activation('relu', name='fc_activation_1')(img_features)
    img_features = layers.Dropout(rate=dropout_rate, name='fc_dropout_1')(img_features)
    img_features = layers.Dense(512, kernel_initializer=DENSE_KERNEL_INITIALIZER, name='fc_2')(img_features)
    img_features = layers.Activation('relu', name='fc_activation_2')(img_features)
    img_features = layers.Dropout(rate=dropout_rate, name='fc_dropout_2')(img_features)
    predictions = layers.Dense(classes,
                               activation='softmax',
                               kernel_initializer=DENSE_KERNEL_INITIALIZER,
                               name = 'prediction')(img_features)

    model = tf.keras.Model(inputs=input, outputs=predictions)

    return model

# Training Function

In [ ]:
def train_model(model, X_train, y_train, X_val, y_val ,epochs, batch_size = None):
      
    callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', patience = 20, restore_best_weights = True)
    
    history = model.fit(x = X_train, y = y_train, epochs = epochs, batch_size = batch_size,
                       shuffle = True, validation_data = (X_val, y_val), callbacks = [callback])
    
    epochs = history.epoch
    hist = pd.DataFrame(history.history)
    
    return epochs, hist

# Model Training

In [ ]:
width_coefficient = 1.0
depth_coefficient = 1.0
default_resolution = 28
dropout_rate = 0.4
learning_rate = 0.00001

EfficientNet = EfficientNet(width_coefficient,
                              depth_coefficient,
                              default_resolution,
                              dropout_rate,
                              global_param,
                              Block_params_list)
EfficientNet.compile(optimizer = tf.keras.optimizers.Adam(lr = learning_rate),
                       loss = 'sparse_categorical_crossentropy',
                       metrics = ['accuracy'])
EfficientNet.summary()

In [ ]:
## Model Test용
batch_size = 1
epochs = 200

epoch, hist = train_model(EfficientNet_1, X_train_data, y_train_data, X_val_data, y_val_data, epochs, batch_size)

list_of_metrics_to_plot = ['loss','val_loss','accuracy','val_accuracy']
loss_plot(epoch, hist, list_of_metrics_to_plot)